In [ ]:
# --- Imports & settings ---
import os, sys, subprocess
from pathlib import Path
import numpy as np
import rasterio
import matplotlib.pyplot as plt

# ================== USER SETTINGS ==================
DSM_TIF = r"C:\Darwin geospatial\25.02 Urban tree risk\Orlando pilot\orlando_dsm_1ft.tif"
OUT_DIR = Path("data")

ANGLES       = range(0, 360, 10)   # 0,10,20,...,350
MAXDIST_KM   = 300
ACCEL        = 1.5
USE_PYRAMIDS = False
KEEP_SAGA_GRIDS = False

# Paths to executables (adjust if needed)
SAGA_CMD = r"C:\Users\usuario\Downloads\saga-9.9.1_x64\saga-9.9.1_x64\saga_cmd.exe"
CANDIDATE_SAGA = [
    os.path.join(os.environ.get("OSGEO4W_ROOT",""), "apps", "saga", "saga_cmd.exe"),
    r"C:\OSGeo4W64\bin\saga_cmd.exe",
    r"C:\Program Files\QGIS 3.34.0\apps\saga\saga_cmd.exe",
    r"C:\Program Files\QGIS 3.34.0\apps\saga-ltr\saga_cmd.exe",
]

GDAL_TRANSLATE = r"C:\Program Files\QGIS 3.34.0\bin\gdal_translate.exe"
CANDIDATE_GDAL = [
    r"C:\OSGeo4W64\bin\gdal_translate.exe",
    r"C:\Program Files\QGIS 3.34.0\bin\gdal_translate.exe",
    r"C:\Program Files\QGIS 3.28.10\bin\gdal_translate.exe",
]

# Fixed PNG canvas (both divisible by 16 → happier ffmpeg)
DPI = 128
WIDTH_PX, HEIGHT_PX = 960, 704  # 960/16=60, 704/16=44

# ================== HELPERS ==================
def which_path(pref, candidates):
    if os.path.isfile(pref): 
        return pref
    for c in candidates:
        if os.path.isfile(c):
            return c
    raise SystemExit(f"Executable not found. Tried: {[pref]+candidates}")

SAGA_CMD = which_path(SAGA_CMD, CANDIDATE_SAGA)
GDAL_TRANSLATE = which_path(GDAL_TRANSLATE, CANDIDATE_GDAL)

def run(cmd):
    """Run a command; raise on failure; echo output on error."""
    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if proc.returncode != 0:
        print(proc.stdout)
        raise RuntimeError(f"Command failed: {' '.join(cmd)}")
    return proc.stdout

def saga_wind_effect(dem_tif, out_base, deg):
    """Compute Wind effect (SAGA) -> .sdat, then convert to GeoTIFF with gdal_translate."""
    sdat = f"{out_base}.sdat"

    # SAGA: ta_morphometry 15 (Wind effect)
    run([
        SAGA_CMD, "ta_morphometry", "15",
        "-DEM", dem_tif,
        "-EFFECT", sdat,
        "-DIR_UNITS", "1",          # degrees
        "-DIR_CONST", str(deg),
        "-MAXDIST", str(MAXDIST_KM),
        "-ACCEL", str(ACCEL),
        "-PYRAMIDS", "1" if USE_PYRAMIDS else "0",
    ])

    # Convert SAGA grid -> GeoTIFF (set NoData at creation)
    tif = f"{out_base}.tif"
    if os.path.exists(tif):  # avoid overwrite issues
        os.remove(tif)
    run([
        GDAL_TRANSLATE,
        "-of", "GTiff",
        "-a_nodata", "-9999",
        "-ot", "Float32",
        "-co", "COMPRESS=DEFLATE",
        "-co", "TILED=YES",
        sdat, tif
    ])
    return tif, sdat

def render_png(raster_path, png_path, title):
    """Percentile-stretched PNG with fixed pixel size (no tight bbox)."""
    with rasterio.open(raster_path) as src:
        arr = src.read(1).astype("float32")
        nodata = src.nodata
        if nodata is not None:
            arr = np.where(arr == nodata, np.nan, arr)

    finite = np.isfinite(arr)
    vmin, vmax = (np.nanpercentile(arr[finite], [2, 98])
                  if finite.any() else (np.nanmin(arr), np.nanmax(arr)))

    plt.figure(figsize=(WIDTH_PX / DPI, HEIGHT_PX / DPI), dpi=DPI)
    im = plt.imshow(arr, vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.axis("off")
    plt.colorbar(im, shrink=0.75, pad=0.02, label="Wind effect (dimensionless)")
    plt.savefig(png_path)  # no tight bbox → identical pixel size
    plt.close()

# ================== RUN (generate PNG frames) ==================
OUT_DIR.mkdir(parents=True, exist_ok=True)
pngs = []

for deg in ANGLES:
    base = str(OUT_DIR / f"windeffect_{deg:03d}")
    print(f"→ Direction {deg:03d}°")
    tif, sdat = saga_wind_effect(DSM_TIF, base, deg)

    png = f"{base}.png"
    render_png(tif, png, f"Wind effect {deg:03d}° (to)")
    pngs.append(png)

    if not KEEP_SAGA_GRIDS:
        for ext in (".sdat", ".sgrd", ".prj", ".mld", ".mgrd"):
            p = f"{base}{ext}"
            if os.path.exists(p):
                try: os.remove(p)
                except: pass

print(f"Frames saved: {len(pngs)} in {OUT_DIR.resolve()}")



→ Direction 000°
→ Direction 010°
→ Direction 020°
→ Direction 030°
→ Direction 040°
→ Direction 050°
→ Direction 060°


In [1]:
# Cell 2
from pathlib import Path
import imageio.v2 as imageio

OUT_DIR = Path("data")
FPS = 8
mp4_path = OUT_DIR / "wind_effect_8fps.mp4"

# Use frames from Cell 1 if available; otherwise glob the folder
try:
    frames_list = pngs
except NameError:
    frames_list = sorted(str(p) for p in OUT_DIR.glob("windeffect_*.png"))

if not frames_list:
    raise SystemExit("No PNG frames found. Run Cell 1 first.")

# Read first frame for size, then stream-write MP4
first = imageio.imread(frames_list[0])
h0, w0 = first.shape[:2]

with imageio.get_writer(
    str(mp4_path),
    fps=FPS,
    codec="libx264",
    quality=8,                 # lower=better quality for imageio; 8 is a good balance
    macro_block_size=16,       # fine since Cell 1 uses 960x704 (divisible by 16)
    ffmpeg_params=["-pix_fmt", "yuv420p"]  # broad player compatibility
) as w:
    w.append_data(first)
    for p in frames_list[1:]:
        im = imageio.imread(p)
        if im.shape[:2] != (h0, w0):
            raise ValueError(f"Frame size mismatch: {p} is {im.shape[:2]}, expected {(h0, w0)}. Re-run Cell 1.")
        w.append_data(im)

print(f"MP4 written: {mp4_path.resolve()}")


MP4 written: C:\Users\usuario\Documents\GitHub\wind_calculator\data\wind_effect_8fps.mp4
